# Terra boilerplate and smart contracts handlers

- First we import all Terra related structures from the sdk and some other packages (`chalk` allows colored console output)
- Next, bind the notebook with local terra (first you need to run it with `docker-compose up` in LocalTerra repository). We recover some default account and give them some names.
- Finally, we wrap some SDK functions into more meaningful code

TODO: better fees control. For the moment fees are hardcoded but it's enough to upload contracts on LocalTerra.


In [13]:
from terra_sdk_wrapper.common_sdk import *

# Contract abstraction and messages declaration

This is my way of abstracting contracts in a framework way. Every contract should inherit the default `Contract` class that implements `instantiate`, `query`, and `execute` methods. 

Children class must implement messages wrapper. For example:

```py
class CW20(Contract):
    def __init__(self, name: str="cw20") -> None:
        self.name = name

    def instantiate_msg(name: str, symbol: str, decimals: int, initial_balances = [], mint = None):
        return {
            "decimals": decimals,
            "name": name,
            "initial_balances": initial_balances,
            "symbol": symbol,
            "mint": mint,
        }
```

The contract will then be called with the following way:

```py
cw20 = CW20()
cw20_init_msg = AnchorToken.instantiate_msg('bsol', 'BSOL', 6, anchor_reward.address, [], {'minter': deployer.key.acc_address})
res = cw20.instantiate(deployer, cw20_code_id, cw20_init_msg)
```

In [14]:
from terra_sdk_wrapper.classes.contract import Contract
from terra_sdk_wrapper.classes.cw20 import CW20

# Instantiation and interaction

Here we can finally play with the contracts.

- First, store the `wasm` on the blockchain
- Second, instantiate the classes
- Third, instantiate the contracts and interact with them

In [ ]:
#============================ Store contracts ============================#

wormhole_token_code_id = store_contract(terra, deployer, 'terraswap_token')
anchor_reward_code_id = store_contract(terra, deployer, 'anchor_bsol_reward')
anchor_token_code_id = store_contract(terra, deployer, 'anchor_bsol_token')
anchor_converter_code_id = store_contract(terra, deployer, 'anchor_bsol_converter')

#======================== Create contract classes ========================#

wormhole_token = WormholeToken("WormholeToken")
anchor_reward = AnchorReward("AnchorReward")
anchor_token = AnchorToken("AnchorToken")
anchor_converter = AnchorConverter("AnchorConverter")

#============================ Init contracts ============================#

anchor_reward_init_msg = AnchorReward.instantiate_msg(deployer.key.acc_address, 'uusd')
res = anchor_reward.instantiate(deployer, anchor_reward_code_id, anchor_reward_init_msg)
anchor_token_init_msg = AnchorToken.instantiate_msg('bsol', 'BSOL', 6, anchor_reward.address, [], {'minter': deployer.key.acc_address})
res = anchor_token.instantiate(deployer, anchor_token_code_id, anchor_token_init_msg)
res = anchor_reward.execute(deployer, AnchorReward.execute_post_initialize_msg(anchor_token.address))
